In [ ]:
import pandas as pd
import numpy as np
import re

train_df = pd.read_csv('data/train.csv')
corpus_df = pd.read_csv('data/corpus.csv')

print(f"Kích thước tập Train ban đầu: {train_df.shape}")
print(f"Kích thước tập Corpus ban đầu: {corpus_df.shape}")
print("\nMẫu cid thô trong train:", train_df['cid'].iloc[7]) 

In [ ]:
train_df.iloc[3].question

In [ ]:
ID = 845
text_1 = corpus_df.iloc[ID].text
cid_1 = corpus_df.iloc[ID].cid
print(cid_1)
print("=" * 50)
print(text_1)

In [ ]:
def clean_cid(cid_str):
    if pd.isna(cid_str):
        return []
    
    cid_str = str(cid_str)
    numbers = re.findall(r'\d+', cid_str)
    return [int(num) for num in numbers]


train_df['cid_list'] = train_df['cid'].apply(clean_cid)


print("Sample sau khi xử lý:")
print(train_df[['cid', 'cid_list']].iloc[7]) # Xem dòng có 2 cid

In [ ]:
valid_cids = set()

for cids in train_df['cid_list']:
    valid_cids.update(cids)

print(f"Số lượng documents (cid) duy nhất được tham chiếu trong tập Train: {len(valid_cids)}")

In [ ]:
corpus_df['cid'] = pd.to_numeric(corpus_df['cid'], errors='coerce')
filtered_corpus_df = corpus_df[corpus_df['cid'].isin(valid_cids)].copy()
filtered_corpus_df.reset_index(drop=True, inplace=True)
print(f"Corpus sau khi lọc: {filtered_corpus_df.shape}")
print(f"Đã loại bỏ {len(corpus_df) - len(filtered_corpus_df)} documents không dùng đến.")


available_cids = set(filtered_corpus_df['cid'].unique())

missing_cids = valid_cids - available_cids

if len(missing_cids) > 0:
    print(f"⚠️ CẢNH BÁO: Có {len(missing_cids)} cid xuất hiện trong Train nhưng KHÔNG có nội dung trong Corpus.")
    print("Các cid bị thiếu (ví dụ 5 cái đầu):", list(missing_cids)[:5])
else:
    print("✅ Dữ liệu sạch: Tất cả cid trong Train đều tìm thấy nội dung trong Corpus.")

In [ ]:
# # Lưu lại để dùng cho bước Chunking/Embedding tiếp theo
# filtered_corpus_df.to_csv('filtered_corpus.csv', index=False)
# train_df.to_csv('processed_train.csv', index=False) # File này đã có cột cid_list dạng list thật

# print("Đã lưu filtered_corpus.csv và processed_train.csv")